In [2]:
import numpy as np

np.set_printoptions(formatter={"float": lambda x: "{:.4f}".format(x)})
import pandas as pd

pd.options.display.float_format = "{:.4f}".format
from scipy.stats import t
import statsmodels.api as sm
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingClassifier

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="darkgrid", context="talk")

diabetes = load_diabetes(as_frame=True)
df = diabetes["data"]
df["target"] = diabetes["target"]
df.info()
# print(diabetes['DESCR'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   age     442 non-null    float64
 1   sex     442 non-null    float64
 2   bmi     442 non-null    float64
 3   bp      442 non-null    float64
 4   s1      442 non-null    float64
 5   s2      442 non-null    float64
 6   s3      442 non-null    float64
 7   s4      442 non-null    float64
 8   s5      442 non-null    float64
 9   s6      442 non-null    float64
 10  target  442 non-null    float64
dtypes: float64(11)
memory usage: 38.1 KB


In [4]:
df.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.0381,0.0507,0.0617,0.0219,-0.0442,-0.0348,-0.0434,-0.0026,0.0199,-0.0176,151.0000
1,-0.0019,-0.0446,-0.0515,-0.0263,-0.0084,-0.0192,0.0744,-0.0395,-0.0683,-0.0922,75.0000
2,0.0853,0.0507,0.0445,-0.0057,-0.0456,-0.0342,-0.0324,-0.0026,0.0029,-0.0259,141.0000
3,-0.0891,-0.0446,-0.0116,-0.0367,0.0122,0.0250,-0.0360,0.0343,0.0227,-0.0094,206.0000
4,0.0054,-0.0446,-0.0364,0.0219,0.0039,0.0156,0.0081,-0.0026,-0.0320,-0.0466,135.0000


Let’s partition the data into training and test sets:

In [9]:
train, test = train_test_split(df, test_size=0.1, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns="target"), df["target"], test_size=0.1, random_state=42
)

x_train = X_train["bmi"]
x_test = X_test["bmi"]

print(f"X_train shape: {X_train.shape}")
print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")
print("\n==============Training data ============")
display(train[["target"]].describe().T)

print(f"X_test shape: {X_test.shape}")
print(f"x_test.shape: {x_test.shape}")
print(f"y_test shape: {y_test.shape}")

print("\n=========== Test data ==================")
test[["target"]].describe().T

X_train shape: (397, 10)
x_train shape: (397,)
y_train shape: (397,)

==============Training data ============


,count,mean,std,min,25%,50%,75%,max
target,397.0000,152.0101,76.9640,25.0000,86.0000,141.0000,209.0000,346.0000


X_test shape: (45, 10)
x_test.shape: (45,)
y_test shape: (45,)

=========== Test data ==================


,count,mean,std,min,25%,50%,75%,max
target,45.0000,153.2222,79.0943,42.0000,90.0000,129.0000,230.0000,310.0000


The target ranges from 25 to 350 with mean of approximately 150 and median around 130–140.

📍 1. Prediction interval
We will now look at three approaches to obtain prediction interval.

💡 1.1. Using standard errors
Let’s build a simple linear regression using bmi to predict the target.

In [21]:
model = LinearRegression()
model.fit(x_train.values.reshape(-1, 1), y_train)
print(f"Intercept: {model.intercept_:.2f}")
print(f"Slope: {model.coef_[0]:.2f}")
print(
    model.predict(x_test.values.reshape(-1, 1))[:5]
)  # predict the first 5 x test values

Intercept: 152.10
Slope: 955.63
[146.1674 187.3669 148.2274 201.7867 132.7776]


We can see the predictions, our best guess. Using the formula below, we can calculate the standard error and obtain prediction intervals:


Variables in green boxes are for the particular observation we are making predictions for while the rest are calculated from the training data.
The formula can be translated into code as follows. We are using a custom object as it allows more flexibility than a function:

In [29]:
t.ppf(1 - 0.1/2, 43)

1.681070701847763

In [32]:
class CustomLinearRegression:
    def __init__(self):
        pass


    ## my note - appears you can assign attributes to self outside of init   
    # this looks like it only applies to simple linear regression
    def fit(self, x, y):
        # Calculate stats
        self.n = len(x)
        self.x_mean = np.mean(x)
        self.y_mean = np.mean(y)
        self.x_gap = (
            x - self.x_mean
        )  # delta between a point value for x and the mean for x
        self.y_gap = y - self.y_mean
        self.ss = np.square(self.x_gap).sum()

        # Find coefficients
        self.slope = np.dot(self.x_gap, self.y_gap) / self.ss
        self.intercept = self.y_mean - self.slope * self.x_mean

        # Find training error
        y_pred = self.intercept + self.slope * x  # get predictions
        self.se_regression = np.sqrt(
            np.square(y - y_pred).sum() / (self.n - 2)
        )  # standard error for regression

    def predict(self, x):
        y_pred = self.intercept + self.slope * x # notice this is same as above 
        return y_pred
    
    def predict_interval(self, x, alpha=0.1):
        t_stat = t.ppf(1 - alpha/2, df=self.n - 2) # t is a Student's t continuous random variable, df=degrees freedom

        # Calculate interval upper and lower boundaries
        df = pd.DataFrame({'x': x})
        for i, value in df['x'].items():
            se = self.se_regression * np.sqrt(
                1 + 1/self.n + np.square(value - self.x_mean)/self.ss
            ) # from formula for prediction interval
            df.loc[i, 'y_pred'] = self.intercept + self.slope * value
            df.loc[i, 'lower'] = df.loc[i, 'y_pred'] - t_stat*se
            df.loc[i, 'upper'] = df.loc[i, 'y_pred'] + t_stat*se
        return df
    
custom_model = CustomLinearRegression()
custom_model.fit(x_train, y_train)
print(f'Intercept: {custom_model.intercept:.2f}')
print(f'Slope: {custom_model.slope:.2f}')
custom_pred = custom_model.predict_interval(x_test)
custom_pred.head()

Intercept: 152.10
Slope: 955.63


,x,y_pred,lower,upper
287,-0.0062,146.1674,42.8550,249.4799
211,0.0369,187.3669,83.9760,290.7578
72,-0.0041,148.2274,44.9162,251.5386
321,0.0520,201.7867,98.3167,305.2567
73,-0.0202,132.7776,29.4435,236.1117


In [34]:
np.mean((custom_pred['upper'] - custom_pred['lower']))

206.94809225846484

Let’s understand this output. In linear regression, predictions represent conditional mean target value. So y_pred, our prediction column, tells us the estimated mean target given the features. Prediction intervals tell us a range of values the target can take for a given record. We can see the lower and upper boundary of the prediction interval from lower and upper columns. This is a 90% prediction interval because we chose alpha=0.1. We will be using the same alpha value for the remainder of this post.

If you are curious, here’re a few ways to interpret prediction intervals:

* There is 90% probability that the actual target value for record 287 will be between 42.8550 and 249.4799.
* We are 90% confident that the actual target value for for record 287 will fall somewhere between 42.8550 and 249.4799 based on their bmi value.
* Approximately 90% of prediction intervals will contain the actual value.

Let’s examine what percentage of target values in the test data were within the prediction intervals: